In [1]:
import warnings
warnings.filterwarnings("ignore", category=Warning)
import pandas as pd
import sys,os

from sgRNA_module import sequencing_primer
import sgRNA_utils.sgRNA_primer_util  as su
from sgRNA_utils.sgRNA_primer_config import config


sys.path.append('/home/yanghe/githubcode/beditor/sgRNA_primer.src')
import sgRNA_utils.sgRNA_primer_util as su
import sgRNA_module.desgin_sgRNA_primer as sr

sys.path.append('/home/yanghe/githubcode/beditor/super_beditor/')
import pipeline 
import pandas as pd
from Bio import SeqIO 
import re  

ModuleNotFoundError: No module named 'sgRNA_module'

In [ ]:
import pandas as pd
df = pd.read_csv(" /hpcfs/fhome/yangchh/editSeqDesign/self/AutoBeditor/beditor/output/yb01/04_offtargets/dofftargets.tsv ")

## 一、操作真核生物的流程

## 1.上传文件
/home/yanghe/primer/editor_gene/temp/Saccharomyces_cerevisiae/

## 2.编写突变信息

In [234]:
df = pd.DataFrame(data=[['NC_001133.9:8706-8706-','G'],
                        ['NC_001133.9:8755-8755-','G'],
                        ['NC_001133.9:8951-8951-','G'],
                        ['NC_001133.9:8979-8979-','G']
                       ]
                  ,columns=['genome coordinate','nucleotide mutation'])
df.to_csv('../../input_output/input_Saccharomyces_cerevisiae.tsv',index=False,sep='\t')  

In [235]:
df "genome_ncbi": "NC_001133.9",   #基因组上位置号    

,genome coordinate,nucleotide mutation
0,NC_001133.9:8706-8706-,G
1,NC_001133.9:8755-8755-,G
2,NC_001133.9:8951-8951-,G
3,NC_001133.9:8979-8979-,G


## 3.编写配置文件

In [253]:
import yaml
data = {
    "dinp": "/home/yanghe/githubcode/beditor/sgRNA_primer/input_output/input_Saccharomyces_cerevisiae.tsv",
    "genome_fna_path":"/home/yanghe/githubcode/beditor/sgRNA_primer/temp/Saccharomyces_cerevisiae/GCF_000146045.2_R64_genomic.fna",
    "genome_gff_path":"/home/yanghe/githubcode/beditor/sgRNA_primer/temp/Saccharomyces_cerevisiae/GCF_000146045.2_R64_genomic.gff",
    "plasmid_gb_path":"/home/yanghe/githubcode/beditor/sgRNA_primer/temp/alsD-28a1.gb",
    
    "reverse_mutations":False,       #'nucleotide wild-type', 'nucleotide mutation'交换顺序
    "host": "Saccharomyces_cerevisiae", #基因组名字
    "genome_release": 'GCF_000204275.1', #基因组版本号
    
    "mutation_format": "nucleotide",  #突变的形式式：氨基酸、核苷酸
    "mutation_type":"N",           #S:突变前的氨基酸与突变后的氨基酸是一样的，N:----不一样  S:一样
    "mutations":"mutations",   

    
    "max_subs_per_codon": 1,       #A--G,A--C,A--T...   一个密码子中突变的核苷酸数量
    "BEs": ['ABE7.9'],  
    "keep_mutation_nonsense": False,    #无意义突变，突变成终止密码子
    "dsubmap_preferred_path":"",   
    "mimetism_level": "medium",
    "pams":['NGG'],
    "non_intermutables": [],  #if ['S','T','K'] is provided all mutations between thsese amino acids are disallowed
    "cores": 6,
    "chunksize":200,
    "bedtools":'bedtools',  #序列提取工具
    "bwa":'bwa',    #序列比对工具
    "samtools":'samtools'   #处理序列比对后的工具
    }
with open('../../input_output/Saccharomyces_cerevisiae.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)  

In [ ]:
import yaml
data = {
    "dinp": "/home/yanghe/githubcode/beditor/sgRNA_primer/input_output/input_Saccharomyces_cerevisiae.tsv",
    "genome_fna_path":"/home/yanghe/githubcode/beditor/sgRNA_primer/temp/Saccharomyces_cerevisiae/GCF_000146045.2_R64_genomic.fna",
    "genome_gff_path":"/home/yanghe/githubcode/beditor/sgRNA_primer/temp/Saccharomyces_cerevisiae/GCF_000146045.2_R64_genomic.gff",
    "plasmid_gb_path":"/home/yanghe/githubcode/beditor/sgRNA_primer/temp/alsD-28a1.gb",
    "host": "Saccharomyces_cerevisiae", #基因组名字
    "genome_release": 'GCF_000204275.1', #基因组版本号
    "mutation_format": "nucleotide",  #突变的形式式：氨基酸、核苷酸
    "BEs": ['ABE7.9'],  
    "pams":['NGG'],
    "bedtools":'bedtools',  #序列提取工具
    "bwa":'bwa',    #序列比对工具
    "samtools":'samtools'   #处理序列比对后的工具
    }
with open('../../input_output/Saccharomyces_cerevisiae.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)  

In [272]:
a ='AGATTTTTCGATGAACAAGAGTATCGGATCAATGAAAAGGAAAGA'

In [273]:
a[21:24]

'TAT'

In [7]:
info = pd.DataFrame(data=[['dinp','突变信息tsv文件的地址'],
                  ['genome_fna_path','基因组fna文件地址'],
                  ['genome_gff_path','基因组gff文件地址'],
                  ['plasmid_gb_path','质粒模板gb文件地址'],
                  ['host','基因组的名字'],
                  ['genome_release','基因组版本号'],
                  ['BEs','可选的碱基编辑器'],
                  ['pams','PAM序列'],
                  ],columns=['配置字段','注释'])

In [8]:
info

,配置字段,注释
0,dinp,突变信息tsv文件的地址
1,genome_fna_path,基因组fna文件地址
2,genome_gff_path,基因组gff文件地址
3,plasmid_gb_path,质粒模板gb文件地址
4,host,基因组的名字
5,genome_release,基因组版本号
6,BEs,可选的碱基编辑器
7,pams,PAM序列


## 4.调用super_beditor

In [4]:
pipeline.pipeline("../../input_output/Saccharomyces_cerevisiae.yml")

INFO:root:bwa index is present
INFO:root:samtools index is present
INFO:root:sizes of contigs are present


['../../input_output/Saccharomyces_cerevisiae/chunks/din_chunk00000001.tsv']
2022_11_15_03_20_22_806353: processing: 1 chunks.


INFO:root:Outputs are located at ../../input_output/Saccharomyces_cerevisiae/05_output


2022_11_15_03_20_22_962769: collecting chunks
2022_11_15_03_20_22_963869: generating outputs


## 5.读取设计出的gRNA信息

In [2]:
sgRNA_df = su.del_Unnamed(pd.read_csv("/home/yanghe/githubcode/beditor/sgRNA_primer/input_output/Saccharomyces_cerevisiae/05_output/doutput.tsv",sep='\t',keep_default_na=False))

## 6.设计相关的引物（sgRNA的oligo、sgRNA重组载体引物、碱基编辑的测序引物）

In [6]:
    
    #加载gb文件读取序列信息
    gb = SeqIO.read('/home/yanghe/githubcode/beditor/sgRNA_primer/temp/alsD-28a1.gb', "genbank")   
    gb_strand_seq = str(gb.seq)
    gb_re_strand_seq = su.revComp(gb_strand_seq)
    gb_seq_tuple = (gb_strand_seq,gb_re_strand_seq)
    
    #加载酶信息
    enzyme_df = pd.read_csv('/home/yanghe/githubcode/beditor/sgRNA_primer/input_output/enzyme.csv') 
    #搜索可用酶
    enzyme_in_plasmid = sr.search_enzyme_from_plasmid(enzyme_df,gb_seq_tuple,cut_num=1)
    #判断是否有可用酶,并根据用户需求选择酶
    final_enzyme = sr.judge_is_not_availability_enzyme(enzyme_in_plasmid,enzyme_name='none')
    assert len(final_enzyme)!=0 
      #将酶与sgRNA_df两个表整合
    sgRNA_df = sr.merge_sgRNA_enzyme(final_enzyme,sgRNA_df)
    #构建sgRNA的oligo
    sgRNA_oligo_df = sr.create_sgRNA_oligo(sgRNA_df)
    #构建sgRNA重组质粒
    sgRNA_plasmid_df=sr.create_sgRNA_plasmid(sgRNA_df,gb_strand_seq)
    #构建重组载体的测序引物
    #用户自定义扩展sgRNA的坐标，0<x<2261,len(plasmid_seg)>y>2284
    x = 2240 
    y = 5000
    plasmid_sequencing_primer_df = sr.create_plasmid_sequencing_primer(x,y,sgRNA_plasmid_df,sequencing_primer)
    
    #构建碱基编辑的测序引物
        #取出基因组上突变位点的前后600bp
    genome_fna_path="/home/yanghe/githubcode/beditor/super_beditor/pub/GCF_000204275.1/fasta/saccharomyces_cerevisiae/dna/genome.fna" 
    sgRNA_plasmid_600bp_df = sr.extract_forward_backward_600bp_from_genome(sgRNA_plasmid_df,genome_fna_path) 
    editor_sequencing_df = sr.create_editor_sequencing_primer(sgRNA_plasmid_600bp_df,util)
    
    #输出
    sheetname2df = {'sgRNA_info':sgRNA_df,'sgRNA_oligo':sgRNA_oligo_df,'editor_sequencing':editor_sequencing_df,'plasmid_sequencing':plasmid_sequencing_primer_df}
    datap = "/home/yanghe/githubcode/beditor/sgRNA_primer/input_output/superBeditor_sgRNA_primer_output.xlsx"
    su.to_excel(sheetname2df,datap,)

guide: id
guide: id


In [4]:
sgRNA_oligo_df

,guide: id,sgRNA_oligo+,sgRNA_oligo-
0,NC_001133.9:8706-8706-|ABE7.9|+|A:G|TAT:TGT|nu...,AGCAGAGTATCGGATCAATGAAA,CCCTTTCATTGATCCGATACTCT
2,NC_001133.9:8951-8951-|ABE7.9|+|A:G|CAG:CGG|nu...,AGCTATTCCAGTAAAGCGGCAGG,CCCCCTGCCGCTTTACTGGAATA
3,NC_001133.9:8979-8979-|ABE7.9|+|A:G|TAC:TGC|nu...,AGCATCCTTACAAAAGACTAAAA,CCCTTTTAGTCTTTTGTAAGGAT


In [5]:
plasmid_sequencing_primer_df

,guide: id,SEQUENCING_PRIMER_1,SEQUENCING_PRIMER_2,SEQUENCING_PRIMER_3,SEQUENCING_PRIMER_4
0,NC_001133.9:8706-8706-|ABE7.9|+|A:G|TAT:TGT|nu...,TTGCTGGCCTTTTGCTCACA,CTGGCTCACAGGCTGATCTT,CAGAGAAAAATCACTCAGGGTCA,GGTCGAGATCCCGGTGCCTA
0,NC_001133.9:8951-8951-|ABE7.9|+|A:G|CAG:CGG|nu...,TTGCTGGCCTTTTGCTCACA,CTGGCTCACAGGCTGATCTT,CAGAGAAAAATCACTCAGGGTCA,GGTCGAGATCCCGGTGCCTA
0,NC_001133.9:8979-8979-|ABE7.9|+|A:G|TAC:TGC|nu...,TTGCTGGCCTTTTGCTCACA,CTGGCTCACAGGCTGATCTT,CAGAGAAAAATCACTCAGGGTCA,GGTCGAGATCCCGGTGCCTA


In [6]:
editor_sequencing_df

,guide: id,SEQUENCING_PRIMER_1,SEQUENCING_PRIMER_2
0,NC_001133.9:8706-8706-|ABE7.9|+|A:G|TAT:TGT|nu...,TGACATATGCAGCACCAACGGT,AAAGCGGCAGGTGGAAGACC
0,NC_001133.9:8951-8951-|ABE7.9|+|A:G|CAG:CGG|nu...,TCTCCACCATGGCCTATCGG,GCACAGTACTACGGGTGCCTCC
0,NC_001133.9:8979-8979-|ABE7.9|+|A:G|TAC:TGC|nu...,GGTCTTCCACCTGCCGCTTT,AGCGCAGTTTTCCGATGGTCCA
